In [1]:
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score

from sklearn.model_selection import KFold
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import auc

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

np.set_printoptions(suppress=True)

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

from IPython.display import clear_output

import json

import warnings

from sklearn import preprocessing

In [2]:
# logistische Regression
LOG_PARAMS = {
    "random_state" : 0,
    "max_iter" : 500,
    "n_jobs" : -1
}

In [3]:
path1 = Path(r"A:\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")
path2 = Path(r"C:\Users\rober\Documents\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")

if path1.is_dir():
    DATASET_DIR = path1
else:
    DATASET_DIR = path2

In [4]:
app_train = pd.read_csv(DATASET_DIR / "4. FillNA" / "application.csv")
bureau = pd.read_csv(DATASET_DIR / "4. FillNA" / "bureau.csv")
pa_pos = pd.read_csv(DATASET_DIR / "4. FillNA" / "pa_pos.csv")
pa_ip = pd.read_csv(DATASET_DIR / "4. FillNA" / "pa_ip.csv")
pa_cc = pd.read_csv(DATASET_DIR / "4. FillNA" / "pa_cc.csv")

In [5]:
app_train = app_train.set_index("SK_ID_CURR")
bureau = bureau.set_index("SK_ID_CURR")
pa_pos = pa_pos.set_index("SK_ID_CURR")
pa_ip = pa_ip.set_index("SK_ID_CURR")
pa_cc = pa_cc.set_index("SK_ID_CURR")

In [6]:
MODEL_APPLICATION = "1.1. Esembler_Logistic_application.json"
MODEL_BUREAU = "1.2. Esembler_Logistic_bureau.json"
MODEL_PA_POS = "1.3. Esembler_Logistic_pa_pos.json"
MODEL_PA_IP = "1.4. Esembler_Logistic_pa_ip.json"
MODEL_PA_CC = "1.5. Esembler_Logistic_pa_cc.json"

In [7]:
with open(DATASET_DIR / "Models" / MODEL_APPLICATION, 'r') as file:
    model_application_data = json.load(file)
    
with open(DATASET_DIR / "Models" / MODEL_BUREAU, 'r') as file:
    model_bureau_data = json.load(file)
    
with open(DATASET_DIR / "Models" / MODEL_PA_POS, 'r') as file:
    model_pa_pos_data = json.load(file)
    
with open(DATASET_DIR / "Models" / MODEL_PA_IP, 'r') as file:
    model_pa_ip_data = json.load(file)
    
with open(DATASET_DIR / "Models" / MODEL_PA_CC, 'r') as file:
    model_pa_cc_data = json.load(file)

In [8]:
print(len(model_application_data["keep"]))
print(len(model_bureau_data["keep"]))
print(len(model_pa_pos_data["keep"]))
print(len(model_pa_ip_data["keep"]))
print(len(model_pa_cc_data["keep"]))

29
8
20
21
22


In [8]:
model_application = LogisticRegression(**LOG_PARAMS)
model_bureau = LogisticRegression(**LOG_PARAMS)
model_pa_pos = LogisticRegression(**LOG_PARAMS)
model_pa_ip = LogisticRegression(**LOG_PARAMS)
model_pa_cc = LogisticRegression(**LOG_PARAMS)

In [9]:
df = app_train[["TARGET"]]
df.head()

,TARGET
SK_ID_CURR,
100002,1.0
100003,0.0
100004,0.0
100006,0.0
100007,0.0


In [10]:
# application prediction

In [11]:
# skalierung der app_train daten

x = app_train[model_application_data["keep"]]

x_col = x.columns
x_idx = x.index

scaler = preprocessing.StandardScaler().fit(x)
x = scaler.transform(x)

x = pd.DataFrame(x)
x.columns = x_col
x.index = x_idx

x.head()

,A_EXT_SOURCE_1,A_DAYS_BIRTH,A_EXT_SOURCE_3,A_EXT_SOURCE_2,A_LANDAREA_AVG,A_AMT_CREDIT,A_CREDIT/INCOME,A_TOTALAREA_MODE,A_DAYS_EMPLOYED,A_CODE_GENDER,A_AMT_ANNUITY,A_FLAG_DOCUMENT_3,A_AMT_INCOME_TOTAL,A_BASEMENTAREA_AVG,A_NONLIVINGAREA_AVG,A_ORGANIZATION_TYPE,A_REGION_POPULATION_RELATIVE,A_REGION_RATING_CLIENT,A_FLAG_EMP_PHONE,A_DAYS_ID_PUBLISH,A_DEF_60_CNT_SOCIAL_CIRCLE,A_NAME_INCOME_TYPE,A_AMT_REQ_CREDIT_BUREAU_QRT,A_DAYS_REGISTRATION,A_CNT_CHILDREN,A_REG_CITY_NOT_WORK_CITY,A_AMT_REQ_CREDIT_BUREAU_YEAR,A_OBS_60_CNT_SOCIAL_CIRCLE,A_OCCUPATION_TYPE
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100002,-2.388187,1.506880,-2.083547,-1.317276,-0.483639,-0.478095,-0.799812,-1.003016,0.929887,-1.388135,-0.166149,0.639065,0.568288,-0.793940,-0.875846,-0.880189,-0.149452,-0.103064,0.468697,0.579154,5.253079,-0.889689,-0.358257,0.379837,-0.577538,-0.547236,-0.513596,0.250544,-0.595378
100003,-1.177997,-0.166821,-0.830909,0.565020,-0.923194,1.725450,0.449521,-0.295394,0.686588,0.720307,0.592669,0.639065,1.499986,-0.508833,-0.335589,-0.787990,-1.252750,-2.067573,0.468697,1.790855,-0.276477,-0.063874,-0.358257,1.078697,-0.577538,-0.547236,-1.087431,-0.170351,-0.334305
100004,0.089223,-0.689509,1.240127,0.217509,0.146687,-1.152888,-0.803353,-0.057224,1.111810,-1.388135,-1.404664,-1.564786,-1.295109,-0.056766,0.008132,-0.695790,-0.783451,-0.103064,0.468697,0.306869,-0.276477,-0.889689,-0.358257,0.206116,-0.577538,-0.547236,-1.087431,-0.591246,-0.595378
100006,0.696588,-0.680114,0.315567,0.712734,0.146138,-0.711430,-0.661414,-0.164791,-0.130739,0.720307,0.177865,0.639065,-0.363410,-0.051043,-0.106653,-0.880189,-0.928991,-0.103064,0.468697,0.369143,-0.276477,-0.889689,-0.013997,-1.375829,-0.577538,-0.547236,0.178116,0.250544,-0.595378
100007,0.128328,-0.892535,0.316786,-1.004048,-0.183289,-0.213734,0.194287,-0.195997,-0.130298,-1.388135,-0.361753,-1.564786,-0.549750,-0.137982,-0.005523,-0.603590,0.563570,-0.103064,0.468697,-0.307263,-0.276477,-0.889689,-0.358257,0.191639,-0.577538,1.827367,-1.087431,-0.591246,-0.334305


In [12]:
y = app_train.loc[app_train.index]["TARGET"]
y.head()

SK_ID_CURR
100002    1.0
100003    0.0
100004    0.0
100006    0.0
100007    0.0
Name: TARGET, dtype: float64

In [13]:
model_application.fit(x,y)

LogisticRegression(max_iter=500, n_jobs=-1, random_state=0)

In [14]:
predicted = pd.DataFrame(model_application.predict_proba(x), index=app_train.index)
predicted.columns = ["A_PAYBACK", "A_DEFAULT"]

In [15]:
df = pd.merge(df, predicted, how="left", left_index=True, right_index=True)
df.head()

,TARGET,A_PAYBACK,A_DEFAULT
SK_ID_CURR,,,
100002,1.0,0.416396,0.583604
100003,0.0,0.718682,0.281318
100004,0.0,0.969872,0.030128
100006,0.0,0.966277,0.033723
100007,0.0,0.924613,0.075387


In [16]:
# bureau prediction

In [17]:
# skalierung der bureau daten

x = bureau[model_bureau_data["keep"]]

x_col = x.columns
x_idx = x.index

scaler = preprocessing.StandardScaler().fit(x)
x = scaler.transform(x)

x = pd.DataFrame(x)
x.columns = x_col
x.index = x_idx

x.head()

,B_Closed,B_AMT_CREDIT_SUM,B_Active,B_AMT_CREDIT_SUM_DEBT,B_CREDIT_DAY_OVERDUE,B_AMT_CREDIT_SUM_OVERDUE,B_DAYS_CREDIT_ENDDATE,B_AMT_ANNUITY
SK_ID_CURR,,,,,,,,
100002,0.744324,-0.572827,-0.031708,-0.460422,-0.032717,-0.033116,-0.462778,-0.393871
100003,-0.142824,1.126252,-0.591045,-0.894294,-0.032717,-0.033116,-0.276204,-0.393871
100004,-0.438540,0.000398,-1.150382,-0.004180,-0.028009,-0.028456,-0.054338,-0.115840
100007,-0.734255,0.000331,-1.150382,-0.003753,-0.026078,-0.026856,-0.048383,-0.103745
100008,-0.438540,-0.416758,-0.591045,0.377006,-0.032717,-0.033116,-0.429454,-0.393871


In [18]:
y = app_train.loc[bureau.index]["TARGET"]
y.head()

SK_ID_CURR
100002    1.0
100003    0.0
100004    0.0
100007    0.0
100008    0.0
Name: TARGET, dtype: float64

In [19]:
model_bureau.fit(x,y)

LogisticRegression(max_iter=500, n_jobs=-1, random_state=0)

In [20]:
predicted = pd.DataFrame(model_bureau.predict_proba(x), index=bureau.index)
predicted.columns = ["B_PAYBACK", "B_DEFAULT"]

In [21]:
df = pd.merge(df, predicted, how="left", left_index=True, right_index=True)
df.head()

,TARGET,A_PAYBACK,A_DEFAULT,B_PAYBACK,B_DEFAULT
SK_ID_CURR,,,,,
100002,1.0,0.416396,0.583604,0.937738,0.062262
100003,0.0,0.718682,0.281318,0.955401,0.044599
100004,0.0,0.969872,0.030128,0.941122,0.058878
100006,0.0,0.966277,0.033723,NaN,NaN
100007,0.0,0.924613,0.075387,0.937121,0.062879


In [22]:
# pa_pos prediction

In [23]:
# skalierung der pa_pos daten

x = pa_pos[model_pa_pos_data["keep"]]

x_col = x.columns
x_idx = x.index

scaler = preprocessing.StandardScaler().fit(x)
x = scaler.transform(x)

x = pd.DataFrame(x)
x.columns = x_col
x.index = x_idx

x.head()

,PA_AMT_CREDIT,PA_AMT_ANNUITY,PA_AMT_APPLICATION,PA_AMT_GOODS_PRICE,PA_CNT_NAME_YIELD_GROUP_high,PA_CNT_NAME_CONTRACT_STATUS_Refused,PA_CNT_CREDITS,PA_POS_CNT_PAYMENTS_LEFT,PA_CNT_PRODUCT_COMBINATION_POS industry with interest,PA_AMT_DOWN_PAYMENT,PA_CNT_NAME_CONTRACT_TYPE_Revolving loans,PA_RATE_DOWN_PAYMENT,PA_CNT_NAME_CLIENT_TYPE_New,PA_CNT_PRODUCT_COMBINATION_Cash X-Sell: low,PA_POS_CNT_DPD,PA_CNT_NAME_YIELD_GROUP_low_normal,PA_CNT_NAME_CLIENT_TYPE_Refreshed,PA_CNT_NAME_PORTFOLIO_Cards,PA_CNT_NAME_PRODUCT_TYPE_walk-in,PA_CNT_CODE_REJECT_REASON_HC
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,
158271,-0.732598,-0.880820,-0.754223,-0.754223,-0.755121,-0.464490,-0.888423,-0.358508,1.100530,-0.707018,-0.543656,-0.869425,0.193456,-0.401357,-0.02179,-0.730497,-0.454386,-0.570431,-0.398699,-0.364585
252457,-0.754666,-0.921917,-0.777095,-0.777095,-0.755121,-0.464490,-0.888423,0.579673,-0.450537,-0.706680,0.445605,-0.868937,-1.622381,-0.401357,-0.02179,-0.730497,1.846507,0.807182,0.513013,-0.364585
260094,3.209171,1.705441,3.038075,3.038075,2.084320,0.085391,3.337410,6.938455,-0.450537,-0.014468,-0.543656,0.855067,0.193456,3.777147,-0.02179,1.564739,0.696060,-0.570431,0.513013,0.360072
176456,-0.635040,-0.645791,-0.629558,-0.629558,-0.755121,-0.464490,-0.888423,-0.358508,1.100530,0.268719,-0.543656,-0.309952,0.193456,-0.401357,-0.02179,-0.730497,-0.454386,-0.570431,-0.398699,-0.364585
256657,1.410781,0.639084,1.285968,1.285968,-0.755121,3.384675,0.168035,2.560277,-0.450537,0.102539,1.434866,2.990206,0.193456,-0.401357,-0.02179,0.799660,-0.454386,2.184795,4.159864,3.258700


In [24]:
y = app_train.loc[pa_pos.index]["TARGET"]
y.head()

SK_ID_CURR
158271    0.0
252457    0.0
260094    0.0
176456    0.0
256657    0.0
Name: TARGET, dtype: float64

In [25]:
model_pa_pos.fit(x,y)

LogisticRegression(max_iter=500, n_jobs=-1, random_state=0)

In [26]:
predicted = pd.DataFrame(model_pa_pos.predict_proba(x), index=pa_pos.index)
predicted.columns = ["PA_POS_PAYBACK", "PA_POS_DEFAULT"]

In [27]:
df = pd.merge(df, predicted, how="left", left_index=True, right_index=True)
df.head()

,TARGET,A_PAYBACK,A_DEFAULT,B_PAYBACK,B_DEFAULT,PA_POS_PAYBACK,PA_POS_DEFAULT
SK_ID_CURR,,,,,,,
100002,1.0,0.416396,0.583604,0.937738,0.062262,0.902993,0.097007
100003,0.0,0.718682,0.281318,0.955401,0.044599,0.982116,0.017884
100004,0.0,0.969872,0.030128,0.941122,0.058878,0.913846,0.086154
100006,0.0,0.966277,0.033723,NaN,NaN,0.934612,0.065388
100007,0.0,0.924613,0.075387,0.937121,0.062879,0.922671,0.077329


In [28]:
# pa_ip prediction

In [29]:
# skalierung der pa_ip daten

x = pa_ip[model_pa_ip_data["keep"]]

x_col = x.columns
x_idx = x.index

scaler = preprocessing.StandardScaler().fit(x)
x = scaler.transform(x)

x = pd.DataFrame(x)
x.columns = x_col
x.index = x_idx

x.head()

,PA_AMT_ANNUITY,PA_AMT_CREDIT,PA_CNT_NAME_CONTRACT_STATUS_Refused,PA_CNT_NAME_YIELD_GROUP_low_normal,PA_CNT_CREDITS,PA_CNT_NAME_CONTRACT_TYPE_Revolving loans,PA_CNT_NAME_PORTFOLIO_Cards,PA_CNT_NAME_CLIENT_TYPE_New,PA_CNT_PRODUCT_COMBINATION_POS industry with interest,PA_CNT_PRODUCT_COMBINATION_Cash X-Sell: low,PA_CNT_NAME_PRODUCT_TYPE_walk-in,PA_IP_DAYS_DIFF_30,PA_AMT_GOODS_PRICE,PA_IP_PAYMENT_180,PA_CNT_NAME_CLIENT_TYPE_Refreshed,PA_IP_DAYS_DIFF_120,PA_AMT_DOWN_PAYMENT,PA_IP_AMT_DIFF_60,PA_IP_DAYS_DIFF_90,PA_IP_DAYS_DIFF_180,PA_IP_PAYMENT_150
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
252457,-0.959737,-0.849221,-0.497596,-0.799024,-0.547812,0.197490,0.525555,-1.536622,-0.460675,-0.479908,0.373464,0.518293,-0.779302,-1.084982,1.823180,-0.298789,-0.324047,-0.008326,-0.537865,0.688666,-1.523831
256657,1.430705,2.059838,3.076558,0.614581,1.156715,1.084066,1.790828,0.161849,-0.460675,-0.479908,3.677278,0.057675,1.856414,-1.211457,-0.458808,-0.194445,2.012909,-0.008326,-0.192590,0.580086,0.159186
198678,1.577307,1.408649,-0.497596,-0.799024,1.156715,-0.689087,-0.739719,0.161849,-0.460675,-0.479908,-0.452490,-1.017101,1.695562,-0.013231,-0.458808,0.605522,1.660798,-0.008326,0.187212,0.652473,-0.148069
394447,-1.106152,-0.868627,-0.497596,-0.799024,-0.547812,0.197490,0.525555,1.860321,-0.460675,-0.479908,0.373464,0.518293,-0.785120,1.413118,-0.458808,0.535960,-0.324682,-0.008326,0.567015,0.688666,-1.306478
156331,0.986910,1.031179,-0.497596,-0.092221,1.156715,1.084066,0.525555,0.161849,-0.460675,-0.479908,-0.452490,0.518293,1.331443,0.710031,-0.458808,0.605522,-0.324682,-0.008326,0.636069,0.326735,1.671529


In [30]:
y = app_train.loc[pa_ip.index]["TARGET"]
y.head()

SK_ID_CURR
252457    0.0
256657    0.0
198678    0.0
394447    0.0
156331    0.0
Name: TARGET, dtype: float64

In [31]:
model_pa_ip.fit(x,y)

LogisticRegression(max_iter=500, n_jobs=-1, random_state=0)

In [32]:
predicted = pd.DataFrame(model_pa_ip.predict_proba(x), index=pa_ip.index)
predicted.columns = ["PA_IP_PAYBACK", "PA_IP_DEFAULT"]

In [33]:
df = pd.merge(df, predicted, how="left", left_index=True, right_index=True)
df.head()

,TARGET,A_PAYBACK,A_DEFAULT,B_PAYBACK,B_DEFAULT,PA_POS_PAYBACK,PA_POS_DEFAULT,PA_IP_PAYBACK,PA_IP_DEFAULT
SK_ID_CURR,,,,,,,,,
100002,1.0,0.416396,0.583604,0.937738,0.062262,0.902993,0.097007,0.921421,0.078579
100003,0.0,0.718682,0.281318,0.955401,0.044599,0.982116,0.017884,NaN,NaN
100004,0.0,0.969872,0.030128,0.941122,0.058878,0.913846,0.086154,NaN,NaN
100006,0.0,0.966277,0.033723,NaN,NaN,0.934612,0.065388,0.958400,0.041600
100007,0.0,0.924613,0.075387,0.937121,0.062879,0.922671,0.077329,0.904789,0.095211


In [34]:
# pa_cc prediction

In [35]:
# skalierung der pa_cc daten

x = pa_cc[model_pa_cc_data["keep"]]

x_col = x.columns
x_idx = x.index

scaler = preprocessing.StandardScaler().fit(x)
x = scaler.transform(x)

x = pd.DataFrame(x)
x.columns = x_col
x.index = x_idx

x.head()

,PA_AMT_ANNUITY,PA_AMT_CREDIT,PA_CC_DPD_30,PA_CC_BALANCE_180,PA_CNT_NAME_CONTRACT_STATUS_Refused,PA_CNT_NAME_YIELD_GROUP_low_normal,PA_CNT_NAME_CLIENT_TYPE_New,PA_CC_BALANCE_60,PA_CNT_NAME_PRODUCT_TYPE_walk-in,PA_CNT_NAME_PORTFOLIO_Cards,PA_CNT_NAME_CONTRACT_TYPE_Revolving loans,PA_CNT_PRODUCT_COMBINATION_POS industry with interest,PA_CC_DPD_60,PA_CC_PAYMENT_60,PA_CNT_NAME_CLIENT_TYPE_Refreshed,PA_CC_DPD_180,PA_CC_PAYMENT_90,PA_CNT_CODE_REJECT_REASON_HC,PA_CC_BALANCE_30,PA_CC_PAYMENT_150,PA_CC_BALANCE_150,PA_CNT_NAME_YIELD_GROUP_high
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,
394447,-0.860134,-0.854669,-0.019318,-0.586986,-0.541101,-0.791255,1.648310,2.614891,0.113888,-0.308747,-0.468764,-0.419765,-0.017627,1.665628,-0.430851,-0.028253,-0.847189,-0.431577,-1.364502,-0.436088,-0.821184,-0.803070
361282,-0.860134,-0.854669,-0.019318,-1.588102,-0.541101,-0.110649,1.648310,1.458173,0.113888,-0.308747,-0.468764,-0.419765,-0.017627,1.839851,-0.430851,-0.028253,0.230048,-0.431577,1.303050,0.386206,-2.104929,-0.803070
436351,-0.860134,-0.854669,-0.019318,0.051803,0.838241,3.292383,-1.366744,-0.126565,0.113888,-0.308747,-0.468764,-0.419765,-0.017627,-0.070003,0.634033,-0.028253,-0.087535,1.417999,0.130616,-0.078649,-0.010999,0.442580
181153,-0.860134,-0.854669,-0.019318,0.051803,0.838241,-0.110649,0.140783,-0.126565,0.113888,-0.308747,-0.468764,-0.419765,-0.017627,-0.070003,0.634033,-0.028253,-0.087535,-0.431577,0.130616,-0.078649,-0.010999,1.065405
309691,-0.860134,-0.854669,-0.019318,0.051803,-0.541101,-0.110649,0.140783,-0.126565,0.113888,-0.308747,-0.468764,-0.419765,-0.017627,-0.070003,-0.430851,-0.028253,-0.087535,-0.431577,0.130616,-0.078649,-0.010999,-0.180245


In [36]:
y = app_train.loc[pa_cc.index]["TARGET"]
y.head()

SK_ID_CURR
394447    0.0
361282    0.0
436351    0.0
181153    0.0
309691    0.0
Name: TARGET, dtype: float64

In [37]:
model_pa_cc.fit(x,y)

LogisticRegression(max_iter=500, n_jobs=-1, random_state=0)

In [38]:
predicted = pd.DataFrame(model_pa_cc.predict_proba(x), index=pa_cc.index)
predicted.columns = ["PA_CC_PAYBACK", "PA_CC_DEFAULT"]

In [39]:
df = pd.merge(df, predicted, how="left", left_index=True, right_index=True)
df.head()

,TARGET,A_PAYBACK,A_DEFAULT,B_PAYBACK,B_DEFAULT,PA_POS_PAYBACK,PA_POS_DEFAULT,PA_IP_PAYBACK,PA_IP_DEFAULT,PA_CC_PAYBACK,PA_CC_DEFAULT
SK_ID_CURR,,,,,,,,,,,
100002,1.0,0.416396,0.583604,0.937738,0.062262,0.902993,0.097007,0.921421,0.078579,NaN,NaN
100003,0.0,0.718682,0.281318,0.955401,0.044599,0.982116,0.017884,NaN,NaN,NaN,NaN
100004,0.0,0.969872,0.030128,0.941122,0.058878,0.913846,0.086154,NaN,NaN,NaN,NaN
100006,0.0,0.966277,0.033723,NaN,NaN,0.934612,0.065388,0.958400,0.041600,0.928554,0.071446
100007,0.0,0.924613,0.075387,0.937121,0.062879,0.922671,0.077329,0.904789,0.095211,NaN,NaN


In [40]:
df = df.fillna(0.5)
df.head()

,TARGET,A_PAYBACK,A_DEFAULT,B_PAYBACK,B_DEFAULT,PA_POS_PAYBACK,PA_POS_DEFAULT,PA_IP_PAYBACK,PA_IP_DEFAULT,PA_CC_PAYBACK,PA_CC_DEFAULT
SK_ID_CURR,,,,,,,,,,,
100002,1.0,0.416396,0.583604,0.937738,0.062262,0.902993,0.097007,0.921421,0.078579,0.500000,0.500000
100003,0.0,0.718682,0.281318,0.955401,0.044599,0.982116,0.017884,0.500000,0.500000,0.500000,0.500000
100004,0.0,0.969872,0.030128,0.941122,0.058878,0.913846,0.086154,0.500000,0.500000,0.500000,0.500000
100006,0.0,0.966277,0.033723,0.500000,0.500000,0.934612,0.065388,0.958400,0.041600,0.928554,0.071446
100007,0.0,0.924613,0.075387,0.937121,0.062879,0.922671,0.077329,0.904789,0.095211,0.500000,0.500000


In [42]:
# finales Modell

In [41]:
model = LogisticRegression(**LOG_PARAMS)

aucs = []

kfold = KFold(5, shuffle=True, random_state=1)
data = df

y = df["TARGET"]
x = df.drop(["TARGET"], axis=1)

for ID_TRAIN, ID_TEST in kfold.split(data):
    
    x_train = x.iloc[ID_TRAIN]
    y_train = y.iloc[ID_TRAIN]
    x_test = x.iloc[ID_TEST]
    y_test = y.iloc[ID_TEST]
    
    model.fit(x_train, y_train)
    auc = roc_auc_score(y_test, model.predict_proba(x_test)[:,1])
    aucs.append(auc)
    
    print('train: %s, test: %s, auc: %s' % (ID_TRAIN, ID_TEST, auc))

print("\n")
print("Durchschnitt: %.4f" % (np.mean(aucs)))
print("Standardabw.: %.2f" % (np.std(aucs)))
print("Varianz:      %.2f" % (np.var(aucs)))

train: [     0      2      3 ... 307508 307509 307510], test: [     1      4      6 ... 307493 307495 307501], auc: 0.7834288984397247
train: [     0      1      2 ... 307508 307509 307510], test: [     3     18     22 ... 307499 307503 307504], auc: 0.7767361837619757
train: [     0      1      2 ... 307508 307509 307510], test: [     9     10     15 ... 307489 307492 307502], auc: 0.7763318791275249
train: [     0      1      2 ... 307506 307507 307508], test: [    12     16     23 ... 307505 307509 307510], auc: 0.7788965499818298
train: [     1      3      4 ... 307505 307509 307510], test: [     0      2      5 ... 307506 307507 307508], auc: 0.7734488834676737


Durchschnitt: 0.78
Standardabw.: 0.00
Varianz:      0.00
